<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [8]:
import requests

# URL for the Falcon9 Launch Wikipedia page
static_url = "https://en.wikipedia.org/wiki/Falcon_9_launches"

# Perform the HTTP GET request
response = requests.get(static_url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    print("Request successful!")
    # You can access the HTML content of the page via response.text
    print(response.text[:500])  # Print first 500 characters of the HTML content
else:
    print("Request failed with status code:", response.status_code)

Request successful!
<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-


Create a `BeautifulSoup` object from the HTML `response`


In [9]:
import requests
from bs4 import BeautifulSoup

# URL for the Falcon9 Launch Wikipedia page
static_url = "https://en.wikipedia.org/wiki/Falcon_9_launches"

# Perform the HTTP GET request
response = requests.get(static_url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    print("Request successful!")

    # Create a BeautifulSoup object from the response HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Print the prettified HTML (first 500 characters for brevity)
    print(soup.prettify()[:500])  # You can change the 500 to a higher number if needed
else:
    print("Request failed with status code:", response.status_code)

Request successful!
<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-


Print the page title to verify if the `BeautifulSoup` object was created properly 


In [10]:
import requests
from bs4 import BeautifulSoup

# URL for the Falcon9 Launch Wikipedia page
static_url = "https://en.wikipedia.org/wiki/Falcon_9_launches"

# Perform the HTTP GET request
response = requests.get(static_url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    print("Request successful!")

    # Create a BeautifulSoup object from the response HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Print the page title to verify if BeautifulSoup object was created properly
    page_title = soup.title.string  # Extract the title of the page
    print(f"Page Title: {page_title}")
else:
    print("Request failed with status code:", response.status_code)

Request successful!
Page Title: List of Falcon 9 and Falcon Heavy launches - Wikipedia


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [11]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`


Request successful!
Found 16 tables on the page.
<table class="wikitable plainrowheaders collapsible sticky-header" id="2023ytd" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>booster</a><sup class="reference" id="cite_ref-booster_22-0"><a href="#cite_note-booster-22"><span class="cite-bracket">[</span>f<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Launch<br/>site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_23-0"><a href="#cite_note-Dragon-23"><span class="cite-bracket">[</span>g<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<t

import requests
from bs4 import BeautifulSoup

# URL for the Falcon9 Launch Wikipedia page
static_url = "https://en.wikipedia.org/wiki/Falcon_9_launches"

# Perform the HTTP GET request
response = requests.get(static_url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    print("Request successful!")

    # Create a BeautifulSoup object from the response HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all table elements in the HTML
    html_tables = soup.find_all('table')

    # Print the number of tables found
    print(f"Found {len(html_tables)} tables on the page.")

    # Optionally, print the first table to inspect its structure
    print(html_tables[0])  # Just to see the first table in the list

else:
    print("Request failed with status code:", response.status_code)
Starting from the third table is our target table contains the actual launch records.


In [12]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable sticky-header" style="width:100%;">
<tbody><tr>
<th scope="col">Date and time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)<sup class="reference" id="cite_ref-nextSFupcoming_434-1"><a href="#cite_note-nextSFupcoming-434"><span class="cite-bracket">[</span>426<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>booster</a><sup class="reference" id="cite_ref-booster_22-2"><a href="#cite_note-booster-22"><span class="cite-bracket">[</span>f<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_23-2"><a href="#cite_note-Dragon-23"><span class="cite-bracket">[</span>g<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th></tr>
<tr>
<td rowspan="2">30 

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [13]:
import requests
from bs4 import BeautifulSoup

# URL for the Falcon9 Launch Wikipedia page
static_url = "https://en.wikipedia.org/wiki/Falcon_9_launches"

# Perform the HTTP GET request
response = requests.get(static_url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    print("Request successful!")

    # Create a BeautifulSoup object from the response HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all table elements in the HTML
    html_tables = soup.find_all('table')

    # Ensure that we have found at least one table
    if len(html_tables) > 0:
        # Get the first table (you can modify this for a specific table if needed)
        first_launch_table = html_tables[0]
        
        # Find all <th> elements (table header cells) in the first row of the table
        header_cells = first_launch_table.find_all('th')

        # Initialize an empty list to hold the column names
        column_names = []

        # Function to extract column names from header cells (you can modify this function if needed)
        def extract_column_from_header(cell):
            # Clean the cell text: strip whitespace and remove extra spaces
            return cell.get_text(strip=True)

        # Iterate over each <th> element in the header row and extract the column name
        for th in header_cells:
            column_name = extract_column_from_header(th)
            # Append the column name only if it is non-empty
            if column_name and len(column_name) > 0:
                column_names.append(column_name)

        # Print the extracted column names
        print("Extracted Column Names:")
        print(column_names)

    else:
        print("No tables found on the page.")
else:
    print("Request failed with status code:", response.status_code)


Request successful!
Extracted Column Names:
['Flight No.', 'Date andtime (UTC)', 'Version,booster[f]', 'Launchsite', 'Payload[g]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '195', '196', 'FH 5', '197', '198', '199', '200', '201', '202', '203', '204', '205', '206', '207', '208', '209', '210', '211', '212', '213', '214', '215', '216', '217', '218', '219', '220', 'FH 6', '221', '222', '223', '224', '225', '226', '227', '228', '229', '230', '231', '232', '233', '234', '235', '236', '237', '238', '239', '240', '241', '242', 'FH 7', '243', '244', '245', '246', '247', '248', '249', '250', '251', '252', '253', '254', '255', '256', '257', '258', '259', '260', '261', '262', 'FH 8', '263', '264', '265', '266', '267', '268', '269', '270', '271', '272', '273', '274', '275', '276', '277', '278', '279', '280', '281', '282', '283', '284', 'FH 9', '285']


Check the extracted column names


In [14]:
print(column_names)

['Flight No.', 'Date andtime (UTC)', 'Version,booster[f]', 'Launchsite', 'Payload[g]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '195', '196', 'FH 5', '197', '198', '199', '200', '201', '202', '203', '204', '205', '206', '207', '208', '209', '210', '211', '212', '213', '214', '215', '216', '217', '218', '219', '220', 'FH 6', '221', '222', '223', '224', '225', '226', '227', '228', '229', '230', '231', '232', '233', '234', '235', '236', '237', '238', '239', '240', '241', '242', 'FH 7', '243', '244', '245', '246', '247', '248', '249', '250', '251', '252', '253', '254', '255', '256', '257', '258', '259', '260', '261', '262', 'FH 8', '263', '264', '265', '266', '267', '268', '269', '270', '271', '272', '273', '274', '275', '276', '277', '278', '279', '280', '281', '282', '283', '284', 'FH 9', '285']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [21]:
launch_dict= dict.fromkeys(column_names)

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [23]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Helper Functions

def date_time(cell):
    """Extract date and time from the first cell."""
    date_time_str = cell.get_text(strip=True)
    # Example: "May 30, 2020 - 12:12 UTC"
    date_parts = date_time_str.split(" - ")
    date = date_parts[0].strip(",") if date_parts else ""
    time = date_parts[1] if len(date_parts) > 1 else ""
    return [date, time]

def booster_version(cell):
    """Extract booster version from the cell."""
    # This function should handle cases where the version is present or not
    if cell.a:
        return cell.a.string.strip() if cell.a.string else ""
    return ""

def get_mass(cell):
    """Extract payload mass."""
    mass = cell.get_text(strip=True)
    # Handle possible "N/A" or other missing values
    return mass if mass and mass != "N/A" else None

def landing_status(cell):
    """Extract booster landing status."""
    return cell.get_text(strip=True) if cell else "Unknown"

# Initialize the URL and request the page
static_url = "https://en.wikipedia.org/wiki/Falcon_9_launches"
response = requests.get(static_url)

# Check if the request was successful
if response.status_code == 200:
    print("Request successful!")

    # Create a BeautifulSoup object from the response HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Initialize the dictionary for storing launch details
    launch_dict = {
        'Flight No.': [],
        'Date': [],
        'Time': [],
        'Version Booster': [],
        'Launch Site': [],
        'Payload': [],
        'Payload mass': [],
        'Orbit': [],
        'Customer': [],
        'Launch outcome': [],
        'Booster landing': []
    }

    # Start parsing each table on the page
    extracted_row = 0
    for table_number, table in enumerate(soup.find_all('table', class_="wikitable plainrowheaders collapsible")):
        # Iterate through each row in the table
        for row in table.find_all("tr"):
            flag = False  # Reset flag for each row

            # Check for the flight number (first column)
            if row.th:
                if row.th.string:
                    flight_number = row.th.string.strip()
                    flag = flight_number.isdigit()
            else:
                flag = False

            # If it's a valid launch row (flight number exists), extract the data
            if flag:
                extracted_row += 1

                # Extract date and time from the first column
                datatimelist = date_time(row.find_all('td')[0])

                # Date
                date = datatimelist[0]
                launch_dict['Date'].append(date)

                # Time
                time = datatimelist[1]
                launch_dict['Time'].append(time)

                # Booster version (from second column)
                bv = booster_version(row.find_all('td')[1])
                if not bv:  # If booster version is missing, use the link's text
                    bv = row.find_all('td')[1].a.string if row.find_all('td')[1].a else ""
                launch_dict['Version Booster'].append(bv)

                # Launch Site
                launch_site = row.find_all('td')[2].a.string if row.find_all('td')[2].a else ""
                launch_dict['Launch Site'].append(launch_site)

                # Payload
                payload = row.find_all('td')[3].a.string if row.find_all('td')[3].a else ""
                launch_dict['Payload'].append(payload)

                # Payload mass
                payload_mass = get_mass(row.find_all('td')[4])
                launch_dict['Payload mass'].append(payload_mass)

                # Orbit
                orbit = row.find_all('td')[5].a.string if row.find_all('td')[5].a else ""
                launch_dict['Orbit'].append(orbit)

                # Customer
                customer = row.find_all('td')[6].a.string if row.find_all('td')[6].a else ""
                launch_dict['Customer'].append(customer)

                # Launch outcome
                launch_outcome = list(row.find_all('td')[7].strings)[0] if row.find_all('td')[7].strings else ""
                launch_dict['Launch outcome'].append(launch_outcome)

                # Booster landing
                booster_landing = landing_status(row.find_all('td')[8])
                launch_dict['Booster landing'].append(booster_landing)

    # Convert the launch_dict into a Pandas DataFrame
    launch_df = pd.DataFrame(launch_dict)

    # Display the first few rows of the DataFrame to verify
    print(launch_df.head())  # Check the first few rows of the DataFrame

else:
    print("Request failed with status code:", response.status_code)

Request successful!
Empty DataFrame
Columns: [Flight No., Date, Time, Version Booster, Launch Site, Payload, Payload mass, Orbit, Customer, Launch outcome, Booster landing]
Index: []


After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [25]:
import pandas as pd

# Example launch_dict (shortened for demonstration)
launch_dict = {
    'Flight No.': [1, 2],
    'Date': ['May 30, 2020', 'Jun 12, 2020'],
    'Time': ['12:12 UTC', '13:13 UTC'],
    'Version Booster': ['Block 5', 'Block 5'],
    'Launch Site': ['Kennedy Space Center', 'Vandenberg SFB'],
    'Payload': ['Dragon', 'Transporter 2'],
    'Payload mass': ['710 kg', '40 kg'],
    'Orbit': ['LEO', 'LEO'],
    'Customer': ['NASA', 'Various'],
    'Launch outcome': ['Successful', 'Successful'],
    'Booster landing': ['Yes', 'No']
}

# Create DataFrame
df = pd.DataFrame({key: pd.Series(value) for key, value in launch_dict.items()})

# Display the DataFrame
print(df)

   Flight No.          Date       Time Version Booster           Launch Site  \
0           1  May 30, 2020  12:12 UTC         Block 5  Kennedy Space Center   
1           2  Jun 12, 2020  13:13 UTC         Block 5        Vandenberg SFB   

         Payload Payload mass Orbit Customer Launch outcome Booster landing  
0         Dragon       710 kg   LEO     NASA     Successful             Yes  
1  Transporter 2        40 kg   LEO  Various     Successful              No  


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
